In [28]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import CSVLogger
from livelossplot import PlotLossesKeras
import os
import numpy as np
from imgaug import augmenters as iaa
import cv2

In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7409746607558104790
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3157314764
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5694250095082357475
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [8]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [40]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")
x_val = np.load("x_val.npy")
y_val = np.load("y_val.npy")
x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

In [10]:
print(x_train.shape, y_train.shape)
print(x_val.shape, x_test.shape, y_val.shape, y_test.shape)

(22046, 200, 200, 3) (22046, 2)
(2756, 200, 200, 3) (2756, 200, 200, 3) (2756, 2) (2756, 2)


In [11]:
IMAGE_SIZE = 200
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
EPOCHS = 30
BATCH_SIZE = 32
TEST_SIZE = 30

In [12]:
TRAINING_LOGS_FILE = "training_logs.csv"
TEST_LOGS_FILE = "test_logs.csv"
MODEL_SUMMARY_FILE = "model_summary.txt"
TEST_FILE = "test_file.txt"
MODEL_FILE = "model.h5"

In [13]:
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

In [14]:
model = Sequential()

In [15]:
model.add(Conv2D(32, 3, 3, border_mode='same', input_shape=input_shape, activation='relu'))
model.add(Conv2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(256, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))
    
model.compile(loss='binary_crossentropy',
            optimizer="adadelta",
            metrics=['accuracy'])
model.summary()

C:\Users\Aiman\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(200, 200,..., activation="relu", padding="same")`
  """Entry point for launching an IPython kernel.
C:\Users\Aiman\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  
C:\Users\Aiman\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  """
C:\Users\Aiman\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  
C:\Users\Aiman\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipyker

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 100, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 50, 50, 128)       73856     
__________

In [17]:
model.fit(x=x_train, y=y_train, validation_data = (x_val,y_val), epochs=EPOCHS, verbose=1, validation_split=0.1)

Train on 22046 samples, validate on 2756 samples
Epoch 1/30
22046/22046 [==============================] - 200s 9ms/step - loss: 0.4691 - acc: 0.7645 - val_loss: 0.1471 - val_acc: 0.9554
Epoch 2/30
22046/22046 [==============================] - 190s 9ms/step - loss: 0.1699 - acc: 0.9476 - val_loss: 0.1380 - val_acc: 0.9608
Epoch 3/30
22046/22046 [==============================] - 190s 9ms/step - loss: 0.1556 - acc: 0.9507 - val_loss: 0.1231 - val_acc: 0.9634
Epoch 4/30
22046/22046 [==============================] - 190s 9ms/step - loss: 0.1469 - acc: 0.9544 - val_loss: 0.1675 - val_acc: 0.9405
Epoch 5/30
22046/22046 [==============================] - 190s 9ms/step - loss: 0.1339 - acc: 0.9590 - val_loss: 0.1149 - val_acc: 0.9663
Epoch 6/30
22046/22046 [==============================] - 190s 9ms/step - loss: 0.1294 - acc: 0.9576 - val_loss: 0.1129 - val_acc: 0.9648
Epoch 7/30
22046/22046 [==============================] - 190s 9ms/step - loss: 0.1237 - acc: 0.9616 - val_loss: 0.1096 - v

In [18]:
model.evaluate(x=x_test, y=y_test, verbose=1)

2756/2756 [==============================] - 8s 3ms/step


[0.4063726602521581, 0.9539187227866474]

In [50]:
# standard augment 
seq1 = iaa.Sequential([
    iaa.Crop(px=(0, 16)), 
    iaa.Fliplr(0.5), 
    iaa.GaussianBlur(sigma=(0, 3.0))
])

# new augment
seq2 = iaa.Sequential([
    iaa.ContrastNormalization((0.5, 1.5)),
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    iaa.Sometimes(0.7, 
        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)
    ),
    iaa.Affine(
        rotate=(-25, 25),
    ),
    iaa.Affine(
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
    ),
    iaa.Affine(
        shear=(-25, 25)
    ),
    
    #iaa.Sometimes(0.8, 
     #   iaa.CoarseDropout(0.03, size_percent=0.1)
    #),
], random_order=True) # apply augmenters in random order


# data augmentation
def augment_data_minimal(x_values, y_values):
    counter = 0
    RESIZE_DIM = 96
    X_values_augmented = []
    Y_values_augmented = []
    for x in x_values:
        for p in range(2):
            
            # seq 1
            Y_values_augmented.append( y_values[counter] )
            images_aug = seq_standard.augment_images(x.reshape(1,RESIZE_DIM,RESIZE_DIM,3))   
            X_values_augmented.append( images_aug.reshape(RESIZE_DIM,RESIZE_DIM,3))
            
            # seq 2
            Y_values_augmented.append( y_values[counter] )
            images_aug = seq_custom.augment_images(x.reshape(1,RESIZE_DIM,RESIZE_DIM,3))   
            X_values_augmented.append( images_aug.reshape(RESIZE_DIM,RESIZE_DIM,3))

        counter = counter + 1
    
    
    # prev number of images = n
    # augmented number of images = n * 4 ( 2 seq 2 times)
    X_values_augmented = np.asarray( X_values_augmented )
    Y_values_augmented = np.asarray( Y_values_augmented )
    return (X_values_augmented, Y_values_augmented)
  
  
  # test time augmentation
def test_of_time(x_test):
  '''
  Arg: x_test: number of test samples of shape (N, w,h,3)
  Returns the predicted labels using test time augmentation    
  '''
 
  pred_label = []
  label = []
  # number of samples in test set
  l = len(x_test)
  for i in range(l):
      # normal sample
      img = x_test[i]
      # augmented sample reshaped(seq acceptd rank-4 tensor) and fed to sequences
      img_a = seq1.augment_images(img.reshape(1,RESIZE_DIM,RESIZE_DIM,3))
      img_b = seq2.augment_images(img.reshape(1,RESIZE_DIM,RESIZE_DIM,3))
      #img_c = seq3.augment_images(img.reshape(1,RESIZE_DIM,RESIZE_DIM,3))

      # augmented sample reshaped back in normal(rank-3 tensor)
      img_a_out = img_a.reshape(RESIZE_DIM,RESIZE_DIM,3)
      img_b_out = img_b.reshape(RESIZE_DIM,RESIZE_DIM,3)
      #img_c_out = img_c.reshape(RESIZE_DIM,RESIZE_DIM,3)

      # test of time!
      y_test1 = model.predict(img_a_out[None,:,:,:])
      y_test2 = model.predict(img_b_out[None,:,:,:])
      #y_test3 = model.predict(img_c_out[None,:,:,:])
      y_test3 = model.predict(img[None,:,:,:])

      # change the params in the weighted average equation for minor changes in the performance
      # future works: arithmetic mean

      # coefficients should add up to 1!
      #y_test = .2*y_test1 + .2*y_test2 + .2*y_test3 + .4*y_test4
      y_test = .2*y_test1 + .2*y_test2 + .6*y_test3
      label.append(y_test)

  # convert to final predicted labels
  pred_labels=np.concatenate(label)
  return pred_labels

In [51]:
label_pred = test_of_time(x_test)
labels= np.array([np.argmax(pred) for pred in label_pred])


ValueError: cannot reshape array of size 120000 into shape (1,96,96,3)

In [47]:
labels.shape

(2756,)

In [46]:
print('Accuracy with TTA: ',np.mean((y_test==labels)))

Accuracy with TTA:  0.0


C:\Users\Aiman\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.
